In [ ]:
pip install transformers

In [2]:
import transformers
transformers.__version__

'4.18.0'

# 1. 모델 로드

In [1]:
from transformers import TFBertForNextSentencePrediction

In [2]:
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForNextSentencePrediction.

All the layers of TFBertForNextSentencePrediction were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


# 2. 토크나이저 로드

In [3]:
from transformers import AutoTokenizer

In [4]:
import tensorflow as tf

In [5]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# 3. 테스트

인덱스 0 : 실제 다음 문장  
인덱스 1 : 서로 상관없는 문장

In [6]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

In [7]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

In [8]:
print(encoding['input_ids'])

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


In [9]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)

[CLS] : 101
[SEP] : 102


In [10]:
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


In [11]:
print(encoding['token_type_ids'])

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


In [12]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

In [13]:
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 6.395877, -6.376609]], dtype=float32)>

In [14]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[9.9999714e-01 2.8377799e-06]], shape=(1, 2), dtype=float32)


In [15]:
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [16]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

In [17]:
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-3.073365 ,  5.9062085]], dtype=float32)>

In [18]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[1.2594071e-04 9.9987400e-01]], shape=(1, 2), dtype=float32)


In [19]:
tf.math.argmax(probs, axis=-1).numpy()

array([1], dtype=int64)